In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls data/csv/processed/record*.csv

data/csv/processed/record-1974.csv data/csv/processed/record-1997.csv
data/csv/processed/record-1975.csv data/csv/processed/record-1998.csv
data/csv/processed/record-1976.csv data/csv/processed/record-1999.csv
data/csv/processed/record-1977.csv data/csv/processed/record-2000.csv
data/csv/processed/record-1978.csv data/csv/processed/record-2001.csv
data/csv/processed/record-1979.csv data/csv/processed/record-2002.csv
data/csv/processed/record-1980.csv data/csv/processed/record-2003.csv
data/csv/processed/record-1981.csv data/csv/processed/record-2004.csv
data/csv/processed/record-1982.csv data/csv/processed/record-2005.csv
data/csv/processed/record-1983.csv data/csv/processed/record-2006.csv
data/csv/processed/record-1984.csv data/csv/processed/record-2007.csv
data/csv/processed/record-1985.csv data/csv/processed/record-2008.csv
data/csv/processed/record-1986.csv data/csv/processed/record-2009.csv
data/csv/processed/record-1987.csv data/csv/processed/record-2010.csv
data/csv/processed/r

In [3]:
files = !ls data/csv/processed/record*.csv
data = pd.concat([pd.read_csv(f, index_col=0) for f in files], keys=files, ignore_index=True)
data.head()

,branch,congressID,ideology_score,page,part,speaker,speech,year
0,Senate,93,-0.296,15,3,Mr. JACKSON.,"Mr. President, there has been much discussi...",1974
1,Senate,93,-0.313,41,3,Mr. RIBICOFF.,"Mr. President, the Subcommittee on Reorgani...",1974
2,Senate,93,0.278,41,3,Mr. TAFT.,"Mr. President, I have be~ come extremely con...",1974
3,Senate,93,-0.056,41,3,Mr. STENNIS.,"Mr. President, the fiscal year 1975 budget f...",1974
4,NaN,93,-0.469,67,3,Mr. PODELL.,"Mr. Speaker, Mr. David Gimbel, widower and f...",1974


In [4]:
remove_speakers = ['The PRESIDING OFFICER.',
                   'The SPEAKER pro tempore.']
data = data[~data.speaker.isin(remove_speakers)]

data['speech'] = data['speech'].str.replace('Mr\.', 'Mr')
data['speech'] = data['speech'].str.replace('Mrs\.', 'Mrs')
data['speech'] = data['speech'].str.replace('Ms\.', 'Ms')
data['speech'] = data['speech'].str.replace('\s+', ' ')
data['speech'] = data['speech'].str.replace('a\.m\. ', 'am ')
data['speech'] = data['speech'].str.replace('p\.m\. ', 'pm ')
data['speech'] = data['speech'].str.replace('H\.R\. ', 'HR ')

data.head()

,branch,congressID,ideology_score,page,part,speaker,speech,year
0,Senate,93,-0.296,15,3,Mr. JACKSON.,"Mr President, there has been much discussion ...",1974
1,Senate,93,-0.313,41,3,Mr. RIBICOFF.,"Mr President, the Subcommittee on Reorganizat...",1974
2,Senate,93,0.278,41,3,Mr. TAFT.,"Mr President, I have be~ come extremely conce...",1974
3,Senate,93,-0.056,41,3,Mr. STENNIS.,"Mr President, the fiscal year 1975 budget for...",1974
4,NaN,93,-0.469,67,3,Mr. PODELL.,"Mr Speaker, Mr David Gimbel, widower and fath...",1974


# Sentences

In [5]:
import re

sentences = pd.concat([data['speech'].str.split('\. ', expand=True)])
df = pd.concat([data, sentences], axis=1). \
    drop('speech', axis=1)
    
df = pd.melt(df, id_vars=['branch','congressID','ideology_score','page','part', 'speaker', 'year'],
             value_name='sentence'). \
    sort_values(by=['year','part', 'page']). \
    reset_index(). \
    drop(['variable', 'index'], axis=1)
    
df['sentence'] = df['sentence'].str.replace('VerDate.*', '')
    
df['sent_length'] = df['sentence'].str.len()
df['caps_length'] = df['sentence'].str.findall('[A-Z]').str.len()
df['letters_length'] = df['sentence'].str.findall('\w').str.len()
df['caps_prop'] = df['caps_length']/df['letters_length']
df = df.query('sent_length > 8 & caps_prop < .4'). \
    drop(['sent_length', 'caps_length', 'letters_length','caps_prop'], axis=1)
    
print(df.shape)
df.head()

(205122, 8)


,branch,congressID,ideology_score,page,part,speaker,year,sentence
0,Senate,93,-0.296,15,3,Mr. JACKSON.,1974,"Mr President, there has been much discussion ..."
1,Senate,93,-0.296,15,3,Mr. JACKSON.,1974,"It is clear that, at some level, prices could ..."
2,Senate,93,-0.296,15,3,Mr. JACKSON.,1974,It is also clear that there is a point at whic...
3,Senate,93,-0.296,15,3,Mr. JACKSON.,1974,Incentives are needed for ex ploration and dev...
4,Senate,93,-0.296,15,3,Mr. JACKSON.,1974,The clear majority of ex pert opinion in Gover...


In [6]:
df.to_csv('data/csv/model/train.csv',index=False)

In [11]:
df.describe()

,congressID,ideology_score,page,year
count,205122.000000,205122.000000,205122.000000,205122.000000
mean,103.641306,-0.038401,437.577695,1994.770863
std,6.461831,0.395162,444.777812,12.894082
min,93.000000,-0.769000,1.000000,1974.000000
25%,98.000000,-0.379000,67.000000,1983.000000
50%,104.000000,-0.186000,223.000000,1995.000000
75%,109.000000,0.340000,783.000000,2006.000000
max,115.000000,0.955000,1634.000000,2018.000000


In [7]:
#pd.set_option('display.max_rows', 500)
#pd.options.display.max_colwidth = 100

#for i in df['sentence'][df['sentence'].str.contains('(.*[0-9]){4}')][50:100]:
#    print(i,'\n')

In [8]:
#df[df['sentence'].str.contains('VerDate')]